<a href="https://colab.research.google.com/github/rgmartin/greece_tourism_project/blob/main/greece_tourism_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Greece tourism detination analysis



## 1. Web scrapping excel files

In [1]:
! pip install plotly --upgrade

In [2]:
#####Step 1: start by importing all of the necessary packages#####
import requests #requesting URLs
import urllib.request #requesting URLs
import os
import time #setting the speed at which the requests run
import re #regexp string manipulation
import pandas as pd #for simplifying data operations (e.g. creating dataframe objects)
import matplotlib.pyplot as plt #for plotting the scraped data
from bs4 import BeautifulSoup #for web-scraping operations
from googletrans import Translator

In [3]:
#####Step 2: connect to the URL in question for scraping#####
url = 'https://insete.gr/perifereies/' 
response = requests.get(url) #Connect to the URL using the "requests" package
response #if successful then it will return 200
#####Step 3: read in the URL via the "BeautifulSoup" package#####
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup)
#####Step 4: filter the HTML object for all link objects######
#link_objs = soup.find_all('a', href=re.compile('/wp-content/uploads'))
excel_links = [link.get('href') for link in soup.find_all('a') if ('xlsx' in link.get('href')) ]

In [4]:
df_parameters= [
                          {'sheet_name': 6, 'header':[2,3],'index_col':[0,1]}
]

In [5]:
def extract_region_name(url):
  file_w_extension = os.path.basename(url)
  file = os.path.splitext(file_w_extension)[0]  
  return ''.join(x for x in file if x.isalpha())

## Origin country and destination regions for incoming tourism in Greece

In [23]:
def read_inbound_tourism_per_country(url, key_figure = 0):
  data = requests.get(url).content
  xl = pd.ExcelFile(data)
  parameters = {'sheet_name': 2, 'header':[3],'index_col':[0,1]}
  # localise those rows corresponding to the first multindex
  df = xl.parse(**parameters)  
  region_name = extract_region_name(url)
  second_level_last_index = list(df.index.get_level_values(1)).index('Σύνολο') 
  df = df.iloc[ :second_level_last_index,key_figure].droplevel(level=0)
  #df.rename(columns={ df.columns[1]: region_name }, inplace = True)



  df.name = region_name
  return df
  
inbound_tourism_per_country_df = pd.concat([read_inbound_tourism_per_country(url) for url in excel_links], axis = 1)

inbound_tourism_per_country_df = inbound_tourism_per_country_df.rename(index = dict(zip(inbound_tourism_per_country_df.index, ['Other', 'Germany', 'Un. Kingdom ','France ','Bulgaria ',
       'Italy', 'Cyprus', 'Northern Macedonia', 'Turkey', 'Netherlands',
       'Poland', 'Albania', 'Romania', 'USA', 'Switzerland', 'Serbia',
       'Belgium', 'Austria', 'USA'])) ) 
print(inbound_tourism_per_country_df)
inbound_tourism_per_country_df = inbound_tourism_per_country_df.loc[inbound_tourism_per_country_df.sum(axis = 1).sort_values(ascending = False).index]
inbound_tourism_per_country_df = inbound_tourism_per_country_df[inbound_tourism_per_country_df.sum().sort_values(ascending = False).index]

                   EasternMacedoniaThrace   Attica  ... Peloponnese CentralGreece
Other                             123.101   75.203  ...         NaN           NaN
Germany                           316.427      NaN  ...         NaN           NaN
Un. Kingdom                        32.999   180.94  ...      46.154        17.099
France                             58.249      NaN  ...         NaN           NaN
Bulgaria                            71.05  667.734  ...     174.383        85.446
Italy                                 NaN  180.257  ...         NaN           NaN
Cyprus                                NaN  200.309  ...      29.348        20.252
Northern Macedonia                    NaN  139.082  ...      28.207         25.17
Turkey                                NaN   81.046  ...         NaN           NaN
Netherlands                           NaN   97.074  ...         NaN           NaN
Poland                                NaN      NaN  ...         NaN           NaN
Albania         

In [18]:
dict(zip(inbound_tourism_per_country_df.index, ['Other', 'Germany', 'In. Kingdom ',' France ',' Bulgaria ',
       'Italy', 'Cyprus', 'Northern Macedonia', 'Turkey', 'Netherlands',
       'Poland', 'Albania', 'Romania', 'USA', 'Switzerland', 'Serbia',
       'Belgium', 'Austria', 'USA']))

{'Αλβανία': 'Albania',
 'Αυστρία': 'Austria',
 'Βέλγιο': 'Belgium',
 'Βουλγαρία': ' Bulgaria ',
 'Βόρεια Μακεδονία': 'Northern Macedonia',
 'Γαλλία': ' France ',
 'Γερμανία': 'Germany',
 'Ελβετία': 'Switzerland',
 'ΗΠΑ': 'USA',
 'ΗΠΑ ': 'USA',
 'Ην. Βασίλειο': 'In. Kingdom ',
 'Ιταλία': 'Italy',
 'Κύπρος': 'Cyprus',
 'Λοιπές': 'Other',
 'Ολλανδία': 'Netherlands',
 'Πολωνία': 'Poland',
 'Ρουμανία': 'Romania',
 'Σερβία': 'Serbia',
 'Τουρκία': 'Turkey'}

In [7]:
inbound_tourism_per_country_df

,Attica,SouthAegean,CentralMacedonia,Crete,IonianIslands,EasternMacedoniaThrace,Peloponnese,Epirus,WesternGreece,Thessaly,CentralGreece,WesternMacedonia,NorthAegean
Turkey,180.257,441.199,249.118,449.847,275.976,123.101,46.154,22.857,40.589,30.377,17.099,13.691,9.549
France,667.734,389.175,361.787,306.255,144.637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bulgaria,200.309,313.248,98.134,149.869,141.648,316.427,28.207,64.924,17.947,25.069,20.252,29.822,5.718
Other,81.046,95.194,208.619,38.104,33.207,71.05,NaN,146.635,91.961,100.961,NaN,21.208,NaN
Germany,180.94,153.583,44.555,149.582,80.039,32.999,29.348,11.389,24.343,13.691,25.17,6.635,1.13
Romania,139.082,22.135,66.704,56.435,48.57,58.249,174.383,19.122,13.127,5.916,85.446,1.226,27.516
Cyprus,75.203,67.753,205.382,47.432,58.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
United Kingdom,97.074,90.184,45.254,38.019,22.481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
import matplotlib.pyplot as plt
import matplotlib
def nodes_df(df):
  regions = list(df.columns)  
  countries = list(df.index)
  nodes = pd.DataFrame(data = regions + countries, columns = ['Label'])  
  nodes.index.name = 'ID'

  cm = plt.get_cmap('tab20')
  n = len(regions  + countries)
  colors = [matplotlib.colors.rgb2hex(rgba) for rgba in cm(range(n))]
  nodes['Color'] = colors

  return nodes  

def links_df(nodes, input_df):

  links = input_df.stack().reset_index()
  links = links.rename(columns = {
      links.columns[0]: 'Origin',
      links.columns[1]: 'Destination',
      links.columns[2]: 'Value',
  })
  links['Origin_id'] = links.apply(lambda row: nodes.index[nodes.Label ==row['Origin']].to_list()[0],axis =1)
  links['Destination_id'] = links.apply(lambda row: nodes.index[nodes.Label ==row['Destination']].to_list()[0],axis =1)  
  links['Color'] = links.apply(lambda row: nodes.Color[nodes.Label ==row['Destination']].values[0],axis =1)  
  
  return links

nodes = nodes_df(inbound_tourism_per_country_df)
links = links_df(nodes, inbound_tourism_per_country_df)

# sort nodes first by outgoing value, then 

import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = nodes['Label'],
      color = list(nodes['Color'])
    ),
    link = dict(
      source = links['Origin_id'], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = links['Destination_id'],
      value = links['Value'],
      color = [f'rgba{tuple(matplotlib.colors.to_rgba(c)[:3] + (0.2,) )}' for c in links['Color']]

  ))])
fig.update_layout( font_size=12,
                  width=800,
    height=600,)
fig.show()

## todo change hover labels
## put appropiate title


                     Label
0                   Attica
1              SouthAegean
2         CentralMacedonia
3                    Crete
4            IonianIslands
5   EasternMacedoniaThrace
6              Peloponnese
7                   Epirus
8            WesternGreece
9                 Thessaly
10           CentralGreece
11        WesternMacedonia
12             NorthAegean
13                  Turkey
14                  France
15                Bulgaria
16                   Other
17                 Germany
18                 Romania
19                  Cyprus
20         United Kingdom 


## Change of inbound tourism shares between Greek regions for the years 2010-2020

In [ ]:
def read_inbound_tourism(url):
  data = requests.get(url).content
  xl = pd.ExcelFile(data)
  parameters = {'sheet_name': 6, 'header':[3],'index_col':[0,1]}
  df = xl.parse(**parameters)
  series = df.loc['Σύνολο'].iloc[0]
  series.name = extract_region_name(url)
  return series

inbound_tourism_df = pd.concat([read_inbound_tourism(url) for url in excel_links], axis = 1)
inbound_tourism_df.index.name = 'Year'
inbound_tourism_df.sort_values(axis=1, by=2020,ascending=False,inplace=True)
inbound_tourism_shares = inbound_tourism_df.div(inbound_tourism_df.sum(axis=1), axis=0)*100
# visualisation based on https://towardsdatascience.com/create-effective-data-visualizations-of-proportions-94b69ad34410
# ideas of visualisation https://www.politico.eu/article/europe-tourism-boom-time-overtourism-top-destinations/
import plotly.express as px
fig = px.bar(inbound_tourism_shares,x = inbound_tourism_shares.index, y =inbound_tourism_shares.columns)
fig.show()

# todo put appropiate labesl and title.

## Relevant questions
* Has the Greece tourism industry been devastated by COVID restrictions?
* How the origin country distribution was affected by COVID?
  - Plot a graph of shares of visitors from different countries (see https://www.kaggle.com/ceshine/impact-of-chinese-tourism-ban-to-taiwan)
* Analysis of seasonality
* Draw insights on how tourism has changed from different countries over time.
* Use other world datasets to draw the possible reasons/factors affecting tourism.
* domestic arrivals vs international air arrivals vs road arrivals

TODO: create function that is able to read any specific data frame set from the excel documents. 
- Compare the share of tourism for each region.!

## Growth rate of tourism spending by international and domestic tourists

## Population of region vs inbound tourism

## Tourism origin country

In [ ]:
- 